<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

### Load Data From CSV File  

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
#!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:


y = df['loan_status'].values

y[0:5]


## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]


# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
# fit a KNN to the training data set using X and Y variables, however as we need to find the best K we are going to split the train set to test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
# checking the size of test and train set
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
# here we are going to find the best K for our data set, for k from 0 to 20 we will fit a model to the training set and check accuracy using test set
# please note that training and test test was created from traing data just to find the best K


from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


Ks = 20
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc



plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 



In [ ]:
# K= 7 is used for the KNN model, we will fit a model to the whole training set using K=7
k = mean_acc.argmax()+1
print( "k=", k)
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)


# Decision Tree

In [ ]:
# in this section we are using Decision Tree model from sklearn to classify the data
from sklearn.tree import DecisionTreeClassifier

In [ ]:
loan_status = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
loan_status # it shows the default parameters
loan_status.fit(X_train,y_train)

In [ ]:
# the decision tree model is loan_status

# Support Vector Machine

In [ ]:
# in this section we are fitting a SVM model to data, we are looking at different kernell options to see which one gives best prediction

from sklearn import svm
clf1 = svm.SVC(kernel='linear')
clf1.fit(X_train, y_train) 

clf2 = svm.SVC(kernel='poly')
clf2.fit(X_train, y_train) 

In [ ]:
clf3 = svm.SVC(kernel='rbf')
clf3.fit(X_train, y_train) 

In [ ]:
clf4 = svm.SVC(kernel='sigmoid')
clf4.fit(X_train, y_train) 

# Logistic Regression

In [ ]:
# in this section we are using Logestic Regression to fit a model to data

from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

In [ ]:
# the logestic regression model is called LR

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()
test_df['loan_status'].value_counts()
test_df.shape

In [ ]:
# Data Visualization and Post_Processing

# convert to date_time object

test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df.head()

#look aat the day of the week people get the loan using the test set

test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df.head()


#convert gategorial featuress to numerical values

test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.head()

# using one hot encoding to convert categorial variables to binary

Feature_test = test_df[['Principal','terms','age','Gender','weekend']]
Feature_test = pd.concat([Feature_test,pd.get_dummies(test_df['education'])], axis=1)
#Feature = pd.concat([Feature,df['education']], axis=1)
Feature_test.drop(['Master or Above'], axis = 1,inplace=True)
# Feature_test.head()

In [ ]:
# shape test set for running the models and predictions

X1_test = Feature_test


y1_test = test_df['loan_status'].values


print(X1_test[0:5])
print(y1_test[0:5])

# normalize Data

X1_test= preprocessing.StandardScaler().fit(X1_test).transform(X1_test)
print(X_test[0:5])

print(X1_test.shape)
print(y1_test.shape)
test_df['loan_status'].value_counts()

In [ ]:
# for KNN, the evaluation is performed here
yhat_neigh = neigh.predict(X1_test)

print("KNN Evaluation")
print("------------------------------------------------")
jacard_KNN =jaccard_similarity_score(y1_test, yhat_neigh)
print("jacard_KNN", jacard_KNN)


f1_score_KNN=f1_score(y1_test, yhat_neigh, average='weighted')
print("f1_score_KNN",f1_score_KNN,'\n')


#for decision tree the evaluation is done here
predTree = loan_status.predict(X1_test)

print("decision tree Evaluation")
print("------------------------------------------------")
jacard_DECTree =jaccard_similarity_score(y1_test, predTree)
print("jacard_DECTree", jacard_DECTree)

f1_score_DECTree=f1_score(y1_test, predTree, average='weighted')
print("f1_score_DECTree",f1_score_DECTree,'\n')



yhat_svm1 = clf1.predict(X1_test)
yhat_svm2 = clf2.predict(X1_test)
yhat_svm3 = clf3.predict(X1_test)
yhat_svm4 = clf4.predict(X1_test)

#print (y_test [0:5])
#print (yhat_svm1 [0:5])
#print (yhat_svm2 [0:5])
#print (yhat_svm3 [0:5])
#print (yhat_svm4 [0:5])


print("SVM Evaluation")
print("------------------------------------------------")
jacard_SVM= jaccard_similarity_score(y1_test, yhat_svm3)
print("jacard_SVM", jacard_SVM)

f1_score_SVM=f1_score(y1_test, yhat_svm3, average='weighted')
print("f1_score_SVM",f1_score_SVM,'\n')



from sklearn.metrics import classification_report

yhat_LR = LR.predict(X1_test)
# to get the count of prediction
#ypd_LR = pd.DataFrame(yhat_LR )


yhat_prob_LR = LR.predict_proba(X1_test)


print("Logestic Regression Evaluation")
print("------------------------------------------------")

jaccard_LR=jaccard_similarity_score(y1_test, yhat_LR)

print("jaccard_LR",jaccard_LR)

#print(classification_report(y1_test, yhat_LR))

f1_score_LR=f1_score(y1_test, yhat_LR, average='weighted')
print("f1_score_LR",f1_score_LR )

log_loss_LR=log_loss(y1_test, yhat_prob_LR)
print("logloss=",log_loss_LR)

#ypd_LR.head(5)
#ypd_LR[0].value_counts()

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | 0.67    | 0.63     | NA      |
| Decision Tree      | 0.72    | 0.74     | NA      |
| SVM                | 0.80    | 0.76     | NA      |
| LogisticRegression | 0.74    | 0.66     | 0.57    |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>